In [ ]:
import re
import numpy as np
import pandas as pd
import geopandas as gpd
from src.download_file import download_file
from src.data_lycees import get_data_lycees
from src.data_lycees import merge_data_lycees
from src.data_biblio import get_data_biblio
from plotnine import *
import folium

pd.set_option('display.width', 120)

In [ ]:
lycees_resultats, lycees_ips, annuaire_education = get_data_lycees()
lycees_data = merge_data_lycees(lycees_resultats, lycees_ips, annuaire_education)
lycees_idf = lycees_data[lycees_data['code_region'] == "11"]


In [ ]:
(
    lycees_data
    .assign(
        # test = lycees_data['taux_men_gnle'] > -200
    )
    .groupby(["code_region", "libelle_region"]).size()
)

In [ ]:
p = (
    ggplot(
        lycees_data,
        aes(
            x = "ips_voie_gt",
            y = "taux_men_gnle",
            color = "va_men_gnle"
    )
  ) + 
  geom_point() + 
  geom_smooth(method = 'lm') +
  scale_color_gradient2() +
    facet_wrap("libelle_region") +
    theme_bw()
)

# p.save(height=8, width=12, format='svg')
p.save("plot_region.png", height=8, width=12, format='png')

In [ ]:
lycees_idf.info()

In [ ]:
(
    lycees_idf
    .assign(
        # test = lycees_data['taux_men_gnle'] > -200
    )
    .groupby(["code_departement", "libelle_departement"]).size()
)

In [ ]:
center = [lycees_idf['latitude'].mean(), lycees_idf["longitude"].mean()]
m = folium.Map(location=center, zoom_start=12)

tooltip = folium.GeoJsonTooltip(
    fields=['libelle_etablissement', 'presents_gnle', 'taux_reu_gnle', 'taux_men_gnle', 'ips_voie_gt'],
    aliases=[
        'Nom :', 'Nombre de candidats présents :', 'Taux de réussite :', 'Taux de mention',
        'Indice de position sociale :'
    ],
    localize=True
)

style_function = lambda x: {
    'fillColor': 'red',
    'color': 'blue',
    'weight': .5,
    'fillOpacity': 0.0
}

folium.GeoJson(
    lycees_idf,
    style_function=style_function,
    tooltip=tooltip
).add_to(m)
m

In [ ]:
biblio = get_data_biblio()
biblio = biblio.dropna(subset=['ADRES_LATITUDE'])
biblio = biblio[biblio['ADRES_PAYS'].isin(["FR", 'fr'])]

In [ ]:
biblio.info()

In [ ]:
biblio.head()

In [ ]:
(
    biblio
    .assign(
        test = biblio['ADRES_PAYS']
    )
    .groupby(["test"]).size()
)

In [ ]:
pd.crosstab(
    biblio['TYPEFAMABES_d'],
    biblio['CONDITIONACCES_d']
)

In [ ]:
lycees_75 = lycees_data[lycees_data['code_departement'] == "075"]
biblio_75 = biblio[biblio['ADRES_CODEPOSTAL'].str[0:2] == "75"]

center = [biblio_75.ADRES_LATITUDE.mean(), biblio_75.ADRES_LONGITUDE.mean()]
m = folium.Map(
    location=center, 
    zoom_start=12
    )

tooltip = folium.GeoJsonTooltip(
    fields=['NOMETABLISSEMENT', 'TYPEETABABES_d', 'CONDITIONACCES_d'],
    aliases=['Nom :', 'TYPEETABABES : ', 'CONDITIONACCES_d :'],
    localize=True
)
folium.GeoJson(
    biblio_75,
    tooltip=tooltip, 
    marker=folium.Marker(icon=folium.Icon(color="green")),
).add_to(m)

tooltip = folium.GeoJsonTooltip(
    fields=['libelle_etablissement', 'presents_gnle', 'taux_reu_gnle', 'taux_men_gnle', 'ips_voie_gt'],
    aliases=[
        'Nom :', 'Nombre de candidats présents :', 'Taux de réussite :', 'Taux de mention',
        'Indice de position sociale :'
    ],
    localize=True
)
folium.GeoJson(
    lycees_75,
    tooltip=tooltip, 
    marker=folium.Marker(icon=folium.Icon(color="blue")),
).add_to(m)

m